In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w9p2/train.csv
/kaggle/input/2024-ml-w9p2/test.csv
/kaggle/input/2024-ml-w9p2/sample_submit.csv


In [2]:
#데이터셋 로드
train = pd.read_csv("/kaggle/input/2024-ml-w9p2/train.csv")
test = pd.read_csv("/kaggle/input/2024-ml-w9p2/test.csv")
submit = pd.read_csv("/kaggle/input/2024-ml-w9p2/sample_submit.csv")

train.head()

,index,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,1,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,2,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,3,6739,6994-KERXL,Male,0,No,No,4,Yes,No,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.5,No
4,4,432,2181-UAESM,Male,0,No,No,2,Yes,No,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5,No


In [3]:
#y 라벨 데이터
y = train['Churn']
train = train.drop(['Churn'], axis=1)

In [4]:
#데이터 열 확인
train.columns

Index(['index', 'Unnamed: 0', 'customerID', 'gender', 'SeniorCitizen',
       'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [5]:
#데이터 shape 확인
train.shape

(4788, 22)

In [6]:
#유의미하지 않은 값 삭제
col = ['index', 'Unnamed: 0', 'customerID', 'gender', 'SeniorCitizen',
       'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

for i in col:
    if (len(np.unique(train[i])) == 4788):
        train = train.drop([i], axis=1)
        test = test.drop([i], axis=1)
        
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            4788 non-null   object 
 1   SeniorCitizen     4788 non-null   int64  
 2   Partner           4788 non-null   object 
 3   Dependents        4788 non-null   object 
 4   tenure            4788 non-null   int64  
 5   PhoneService      4788 non-null   object 
 6   MultipleLines     4788 non-null   object 
 7   InternetService   4788 non-null   object 
 8   OnlineSecurity    4788 non-null   object 
 9   OnlineBackup      4788 non-null   object 
 10  DeviceProtection  4788 non-null   object 
 11  TechSupport       4788 non-null   object 
 12  StreamingTV       4788 non-null   object 
 13  StreamingMovies   4788 non-null   object 
 14  Contract          4788 non-null   object 
 15  PaperlessBilling  4788 non-null   object 
 16  PaymentMethod     4788 non-null   object 


In [7]:
#object 타입 실수형으로 변환
col = ['gender', 
       'Partner', 'Dependents',  'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod']

from sklearn.preprocessing import LabelEncoder

for i in col:
    le = LabelEncoder()
    train[i] = le.fit_transform(train[i].values)
    test[i] = le.transform(test[i].values)

In [8]:
#확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            4788 non-null   int64  
 1   SeniorCitizen     4788 non-null   int64  
 2   Partner           4788 non-null   int64  
 3   Dependents        4788 non-null   int64  
 4   tenure            4788 non-null   int64  
 5   PhoneService      4788 non-null   int64  
 6   MultipleLines     4788 non-null   int64  
 7   InternetService   4788 non-null   int64  
 8   OnlineSecurity    4788 non-null   int64  
 9   OnlineBackup      4788 non-null   int64  
 10  DeviceProtection  4788 non-null   int64  
 11  TechSupport       4788 non-null   int64  
 12  StreamingTV       4788 non-null   int64  
 13  StreamingMovies   4788 non-null   int64  
 14  Contract          4788 non-null   int64  
 15  PaperlessBilling  4788 non-null   int64  
 16  PaymentMethod     4788 non-null   int64  


In [9]:
#TotalCharges float 타입으로 변환
#train['TotalCharges'] = pd.to_numeric(train['TotalCharges'], downcast='float')

In [10]:
#280번째 데이터 확인
train['TotalCharges'][280]

' '

In [11]:
#train에 ' '로 채워진 데이터 삭제
drop_idx = train[train['TotalCharges'] == " "].index
drop_idx

Index([280, 512, 2213, 2470, 2617, 3457, 4297, 4303, 4555, 4759], dtype='int64')

In [12]:
#TotalCharges 데이터가 비어있는 데이터 삭제
train = train.drop(drop_idx, axis=0)
y = y.drop(drop_idx)

In [13]:
#TotalCharges float 타입으로 변환
train['TotalCharges'] = train['TotalCharges'].astype('float64')

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4778 entries, 0 to 4787
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            4778 non-null   int64  
 1   SeniorCitizen     4778 non-null   int64  
 2   Partner           4778 non-null   int64  
 3   Dependents        4778 non-null   int64  
 4   tenure            4778 non-null   int64  
 5   PhoneService      4778 non-null   int64  
 6   MultipleLines     4778 non-null   int64  
 7   InternetService   4778 non-null   int64  
 8   OnlineSecurity    4778 non-null   int64  
 9   OnlineBackup      4778 non-null   int64  
 10  DeviceProtection  4778 non-null   int64  
 11  TechSupport       4778 non-null   int64  
 12  StreamingTV       4778 non-null   int64  
 13  StreamingMovies   4778 non-null   int64  
 14  Contract          4778 non-null   int64  
 15  PaperlessBilling  4778 non-null   int64  
 16  PaymentMethod     4778 non-null   int64  
 17  

In [15]:
#random seed 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [16]:
#로지스틱 회귀
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=seed, max_iter=1000).fit(train, y)

#train 예측 및 정확도 출력
train_pred = logit.predict(train)

from sklearn.metrics import accuracy_score
print(accuracy_score(train_pred, y))

0.8061950606948514


In [17]:
#test 데이터 예측
test_pred = logit.predict(test)
print(test_pred)
submit.info()

['No' 'No' 'No' ... 'No' 'No' 'Yes']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   1198 non-null   int64  
 1   Churn   0 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 18.8 KB


In [18]:
#yes=1, no=0
test_pred = pd.Series(test_pred)
test_pred.replace({'Yes':1, 'No':0}, inplace=True)

/tmp/ipykernel_18/722162499.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pred.replace({'Yes':1, 'No':0}, inplace=True)


In [19]:
print(test_pred)

0       0
1       0
2       0
3       1
4       0
       ..
1193    0
1194    0
1195    0
1196    0
1197    1
Length: 1198, dtype: int64


In [20]:
#submit 확인
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   1198 non-null   int64  
 1   Churn   0 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 18.8 KB


In [21]:
#submit 저장
submit['Churn'] = test_pred
submit.to_csv('submit.csv', index=False)
submit.head()

,index,Churn
0,5,0
1,10,0
2,13,0
3,18,1
4,20,0
